In [1]:
# work on tensorflow quick start for experts
import tensorflow as tf

In [3]:
# import dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# normalize the image data
x_train, x_test = x_train/255, x_test/255

In [4]:
# add channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [6]:
# create dataset from slices and split them into batches
training_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [76]:
# create model class by subclassing the keras model API

class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(32, 3, activation="relu")
        self.flatten = tf.keras.layers.Flatten()
        self.d1 = tf.keras.layers.Dense(128, activation="relu")
        self.batch_norm = tf.keras.layers.BatchNormalization(axis=1)
        self.d2 = tf.keras.layers.Dense(10)
    
    
    def call(self, inputs, training=None, mask=None):
        x = self.conv1(inputs)
        x = self.flatten(x)
        x = self.d1(x)
        # perform batch normalization if training
        if training:
            x = self.batch_norm(x)
        x = self.d2(x)
        return x

# # dont use the output parameter as it is already being used by the tensorflow framework
# myModel = MyModel()

# class MyModel(tf.keras.Model):
#     def __init__(self):
#         super(MyModel, self).__init__()
#         self.conv1 = tf.keras.layers.Conv2D(32, 3, activation='relu')
#         self.flatten = tf.keras.layers.Flatten()
#         self.d1 = tf.keras.layers.Dense(128, activation='relu')
#         self.d2 = tf.keras.layers.Dense(10)

#     def call(self, x):
#         x = self.conv1(x)
#         x = self.flatten(x)
#         x = self.d1(x)
#         return self.d2(x)

# Create an instance of the model
model = MyModel()

In [77]:
# instantiate loss and optimizer
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.legacy.Adam() # using legacy adam optimizer becuase the current implementation of the adam optimizer does not perform well on mac books

In [78]:
# create training loss and accuracy
training_loss = tf.keras.metrics.Mean(name='training_loss')
training_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='training_accuracy')

# create test loss and accuracy
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [79]:
@tf.function
def training_step(model, loss_fn, optimizer, training_loss, training_accuracy, x_train, y_train):
    with tf.GradientTape() as tape:
        logits = model(x_train, training=True)
        loss = loss_fn(y_train, logits)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    # add training loss and accuracy
    training_loss(loss)
    training_accuracy(y_train, logits)

In [80]:
@tf.function
def test_step(model, loss_fn, test_loss, test_accuracy, x_test, y_test):
    logits = model(x_test, training=False)
    loss = loss_fn(y_test, logits)
    # add test loss and accuracy
    test_loss(loss)
    test_accuracy(y_test, logits)

In [81]:
num_epochs = 5
for epoch in range(1, num_epochs+1):
    # reset the states of all the metrics
    training_accuracy.reset_states()
    training_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    for image, labels in training_ds:
        training_step(model, loss, optimizer, training_loss, training_accuracy, image, labels)
    
    for image, labels in test_ds:
        test_step(model, loss,  test_loss, test_accuracy, image, labels)
    
    # .result is used for metrics
    print(
        f'Epoch {epoch}, '
        f'Loss: {training_loss.result()}, '
        f'Accuracy: {training_accuracy.result() * 100}, '
        f'Test Loss: {test_loss.result()}, '
        f'Test Accuracy: {test_accuracy.result() * 100}'
    )

Epoch 1, Loss: 0.13008859753608704, Accuracy: 96.06500244140625, Test Loss: 0.5512877106666565, Test Accuracy: 86.48999786376953
Epoch 2, Loss: 0.08879796415567398, Accuracy: 98.52333068847656, Test Loss: 0.38938674330711365, Test Accuracy: 92.83000183105469
Epoch 3, Loss: 0.06787770986557007, Accuracy: 99.17333221435547, Test Loss: 0.43541091680526733, Test Accuracy: 88.2300033569336
Epoch 4, Loss: 0.0548548549413681, Accuracy: 99.49500274658203, Test Loss: 0.29823029041290283, Test Accuracy: 92.1500015258789
Epoch 5, Loss: 0.04661491885781288, Accuracy: 99.52832794189453, Test Loss: 0.2038724571466446, Test Accuracy: 95.26000213623047


In [2]:
from queue import Queue

my_queue = Queue(maxsize=3)

In [6]:
my_queue.put(1)
